In [1]:
%pylab inline
%load_ext autoreload
%autoreload 2


Populating the interactive namespace from numpy and matplotlib


## Download data on Outfall Locations
From [here](https://www.fisheries.noaa.gov/inport/item/66706)

We'll try to visualize just a scatter of all outfall locations by geo

In [4]:
import pandas as pd
import fiona

# No need to pass "layer='etc'" if there's only one layer
df = []
locs = {'type': 'FeatureCollection'}
locs['features'] = []
with fiona.open('./WastewaterOutfall.gpkg') as layer:
    for feature in layer:
        df.append({
            'id': feature['id'],
            'major_minor': feature['properties']['majorMinorStatus'],
            'city_state': f"{feature['properties']['cityName']}, {feature['properties']['stateUSPS']}",
            "lat": feature['geometry']['coordinates'][1],
            "long": feature['geometry']['coordinates'][0],
            "permit_status": feature['properties']['permitStatus'],
            "text": f"{feature['properties']['cityName']}, {feature['properties']['stateUSPS']}: {feature['properties']['majorMinorStatus']}; Permit \n{feature['properties']['permitStatus']}",            
        })
        feature['properties']['cityStateName'] = f"{feature['properties']['cityName']}, {feature['properties']['stateUSPS']}"
        locs['features'].append(feature)
df = pd.DataFrame(df)
# locs['features'] = features
#         print(feature['geometry'], feature['properties']['cityName'])

df['point_score'] = 100
df.loc[df.major_minor == 'Minor', 'point_score'] = 50

df['point_color'] = 'blue'
df.loc[df.major_minor == 'Minor', 'point_color'] = 'red'

In [5]:
df.head()

,id,major_minor,city_state,lat,long,permit_status,text,point_score,point_color
0,1,Minor,"Cambridge, MA",42.363464,-71.079669,Admin Continued,"Cambridge, MA: Minor; Permit \nAdmin Continued",50,red
1,2,Minor,"Stratford, CT",41.163690,-73.128337,Expired,"Stratford, CT: Minor; Permit \nExpired",50,red
2,3,Major,"Stamford, CT",41.045462,-73.528842,Effective,"Stamford, CT: Major; Permit \nEffective",100,blue
3,4,Minor,"Bridgewater, MA",41.995350,-70.967080,Effective,"Bridgewater, MA: Minor; Permit \nEffective",50,red
4,5,Minor,"Bath, ME",43.927253,-69.810980,Effective,"Bath, ME: Minor; Permit \nEffective",50,red


In [ ]:
import plotly.express as px
import plotly.io as pio
import plotly.graph_objects as go

pio.renderers.default = "browser"

fig = go.Figure(data=go.Scattergeo(
    lon = df['long'],
    lat = df['lat'],
    text = df['text'],
    locationmode="USA-states",
    marker_color=df['point_color'],
    hoverinfo='text'
    ))
fig.update_layout(
    title = 'Outfall locations',
    geo_scope='usa',
    )

fig.show()

## Glodap data
(Desal data not by lat long, so ignoring for now)

Obtained from [here](https://www.ncei.noaa.gov/access/metadata/landing-page/bin/iso?id=gov.noaa.nodc:0162565), via the Olsen et al 2017 reference in [this section](https://bg.copernicus.org/articles/20/27/2023/#section3) in Michael Tyka's paper on Limits of near-coast OAE

- Downloaded via FTP
- navigate to `data-product/GLODAPv2 Merged Master File.csv.zip` and unzip


In [10]:
dfg = pd.read_csv("./oae-dash/GLODAPv2 Merged Master File.csv")

In [17]:
pd.set_option('max_columns', 1000)

In [28]:
# dfg.year.max()
dfg.query('year == "2013"')

,cruise,station,cast,year,month,day,hour,minute,latitude,longitude,bottomdepth,maxsampdepth,bottle,pressure,depth,temperature,theta,salinity,salinityf,salinityqc,sigma0,sigma1,sigma2,sigma3,sigma4,gamma,oxygen,oxygenf,oxygenqc,aou,aouf,nitrate,nitratef,nitrateqc,nitrite,nitritef,silicate,silicatef,silicateqc,phosphate,phosphatef,phosphateqc,tco2,tco2f,tco2qc,talk,talkf,talkqc,phts25p0,phts25p0f,phtsinsitutp,phtsinsitutpf,phtsqc,cfc11,pcfc11,cfc11f,cfc11qc,cfc12,pcfc12,cfc12f,cfc12qc,cfc113,pcfc113,cfc113f,cfc113qc,ccl4,pccl4,ccl4f,ccl4qc,sf6,psf6,sf6f,c13,c13f,c14,c14f,c14err,h3,h3f,h3err,he3,he3f,he3err,he,hef,heerr,neon,neonf,neonerr,o18,o18f,toc,tocf,doc,docf,don,donf,tdn,tdnf,chla,chlaf
252799,235,1.0,1,2013,3,23,11,43,-50.7090,-57.0020,346.624,332.2,1,331.6,328.0,4.5240,4.49946,34.0753,2,1,26.9980,31.5893,36.0778,40.4653,44.7541,27.1388,275.451,2,1,37.7446,2,24.9125,0,1,-9999.0,9,12.86220,0,1,-9999.00,9,1,-9999.0,9,1,2269.79,2,1,-9999.00000,9,-9999.00000,9,1,-9999.0,-9999.0,9,1,-9999.0,-9999.0,9,1,-9999.0,-9999.0,9,1,-9999.0,-9999.0,9,1,-9999.0,-9999.0,9,-9999.0,9,-9999.0,9,-9999.0,-9999.0,9.0,-9999.0,-9999.0,9,-9999.0,-9999.0,9,-9999.0,-9999.0,9,-9999.0,-9999.0,9,-9999.0,9,-9999.0,9,-9999.0,9,-9999.0,9,-9999.0,9
252800,235,1.0,1,2013,3,23,11,43,-50.7090,-57.0020,346.624,332.2,2,332.2,329.0,4.5136,4.48904,34.0760,2,1,26.9997,31.5913,36.0800,40.4678,44.7568,27.1409,274.861,2,1,38.4110,2,24.9400,2,1,-9999.0,9,12.95560,2,1,-9999.00,9,1,-9999.0,9,1,2269.91,2,1,-9999.00000,9,-9999.00000,9,1,-9999.0,-9999.0,9,1,-9999.0,-9999.0,9,1,-9999.0,-9999.0,9,1,-9999.0,-9999.0,9,1,-9999.0,-9999.0,9,-9999.0,9,-9999.0,9,-9999.0,-9999.0,9.0,-9999.0,-9999.0,9,-9999.0,-9999.0,9,-9999.0,-9999.0,9,-9999.0,-9999.0,9,-9999.0,9,-9999.0,9,-9999.0,9,-9999.0,9,-9999.0,9
252801,235,1.0,1,2013,3,23,11,43,-50.7090,-57.0020,346.624,332.2,3,300.0,297.0,4.9767,4.95353,34.0292,2,1,26.9110,31.4911,35.9686,40.3456,44.6241,27.0356,280.487,2,1,29.4496,2,23.8000,2,1,-9999.0,9,9.34920,2,1,-9999.00,9,1,2135.8,0,1,2265.73,2,1,7.70605,2,7.99217,2,1,-9999.0,-9999.0,9,1,-9999.0,-9999.0,9,1,-9999.0,-9999.0,9,1,-9999.0,-9999.0,9,1,-9999.0,-9999.0,9,-9999.0,9,-9999.0,9,-9999.0,-9999.0,9.0,-9999.0,-9999.0,9,-9999.0,-9999.0,9,-9999.0,-9999.0,9,-9999.0,-9999.0,9,-9999.0,9,-9999.0,9,-9999.0,9,-9999.0,9,-9999.0,9
252802,235,1.0,1,2013,3,23,11,43,-50.7090,-57.0020,346.624,332.2,4,300.7,298.0,4.9705,4.94729,34.0303,2,1,26.9126,31.4928,35.9705,40.3476,44.6262,27.0375,280.595,2,1,29.3849,2,23.8124,0,1,-9999.0,9,9.37026,0,1,-9999.00,9,1,-9999.0,9,1,2265.83,2,1,-9999.00000,9,-9999.00000,9,1,-9999.0,-9999.0,9,1,-9999.0,-9999.0,9,1,-9999.0,-9999.0,9,1,-9999.0,-9999.0,9,1,-9999.0,-9999.0,9,-9999.0,9,-9999.0,9,-9999.0,-9999.0,9.0,-9999.0,-9999.0,9,-9999.0,-9999.0,9,-9999.0,-9999.0,9,-9999.0,-9999.0,9,-9999.0,9,-9999.0,9,-9999.0,9,-9999.0,9,-9999.0,9
252803,235,1.0,1,2013,3,23,11,43,-50.7090,-57.0020,346.624,332.2,5,203.3,201.0,5.4916,5.47524,33.9656,2,1,26.7998,31.3672,35.8324,40.1975,44.4645,26.9067,281.218,2,1,25.0748,2,22.4200,2,1,-9999.0,9,6.97760,2,1,-9999.00,9,1,2128.7,0,1,2265.08,2,1,7.72354,2,8.00648,2,1,-9999.0,-9999.0,9,1,-9999.0,-9999.0,9,1,-9999.0,-9999.0,9,1,-9999.0,-9999.0,9,1,-9999.0,-9999.0,9,-9999.0,9,-9999.0,9,-9999.0,-9999.0,9.0,-9999.0,-9999.0,9,-9999.0,-9999.0,9,-9999.0,-9999.0,9,-9999.0,-9999.0,9,-9999.0,9,-9999.0,9,-9999.0,9,-9999.0,9,-9999.0,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995155,721,1240880.0,1,2013,11,13,3,1,64.3383,-27.9775,1045.300,1035.3,7,201.3,199.0,7.8150,7.79485,35.1219,2,0,27.4006,31.9036,36.3061,40.6104,44.8188,-9999.0000,272.100,2,0,15.8679,2,13.8500,2,0,-9999.0,9,5.85000,2,0,0.97

### Simple plotly scattergeo

In [39]:
import plotly.express as px
import plotly.io as pio
import plotly.graph_objects as go

pio.renderers.default = "browser"

dfgy = dfg.query("year >= 2009")
fig = go.Figure(data=go.Scattergeo(
    lon = dfgy['longitude'],
    lat = dfgy['latitude'],
#     text = dfgy['talk'],
#     locationmode="USA-states",
    marker_color=dfgy['talk'],
#     hoverinfo='text'
    ))
fig.update_layout(
    title = 'Total alkalinity',
    geo_scope='usa',
    )

fig.show()

### Mapbox scatter -- bit better for visualizations and faster zooms

In [109]:
# from configparser import ConfigParser
# config_parser = ConfigParser()
# config = config_parser.read('oae-dash/config.ini')
# mapbox_token = config['mapbox']['secret_token']

mapbox_token = 'pk.eyJ1IjoiY3J1ZHJhdmVuIiwiYSI6ImNsY3hudzJ2bjBiMDYzcG9kanBlbzAyN2gifQ.T_YCJafqnnVPafhfKD83ag'
dfgy = dfg.query("year >= 2011").replace(-9999, nan).groupby([
    'latitude', 'longitude', 'year', 'month', 'day'
    ]).mean().reset_index()
dfgy['date'] = pd.to_datetime(dict(year=dfgy.year, month=dfgy.month, day=dfgy.day))

fig = go.Figure(
    go.Scattermapbox(
        lat=dfgy['latitude'],
        lon=dfgy['longitude'],
        mode='markers',
        marker=go.scattermapbox.Marker(
            size=log1p(dfgy['talk'].replace(nan, 0)),
            opacity=0.5,
            color=dfgy['tco2'].replace(-9999, nan),
            showscale=True,
            colorbar={'title':'TCO2', 'titleside':'top', 'thickness':4, 'ticksuffix':' mu mol / kg'},
        ),
        customdata=np.stack((
           dfgy['date'].astype(str),
           dfgy['tco2'],
           dfgy['talk'],
           dfgy['salinity']
        ), axis=-1),

        hovertemplate="""
<extra></extra>
%{customdata[0]}<br>
TCO2: %{customdata[1]:.3f} mu mol/kg<br>  
ALK: %{customdata[2]:.3f}<br>
SAL: %{customdata[3]:.3f}<br>
""",
    )
)

# Specify layout information
fig.update_layout(
    mapbox=dict(
        accesstoken=mapbox_token, #
        center=go.layout.mapbox.Center(lat=45, lon=-73),
        zoom=1
    )
)
fig.show()

### Mapbox animation so we can see info by year / month

In [138]:
pd.options.plotting.backend = 'plotly'

In [173]:
dfgy['talk'].hist()
# yscale('log')

In [169]:
mapbox_token = 'pk.eyJ1IjoiY3J1ZHJhdmVuIiwiYSI6ImNsY3hudzJ2bjBiMDYzcG9kanBlbzAyN2gifQ.T_YCJafqnnVPafhfKD83ag'
dfgy = dfg.query("year >= 2008").replace(-9999, nan).groupby([
    'latitude', 'longitude', 'year', 'month' #, 'day'
    ]).mean().reset_index()
dfgy['date'] = pd.to_datetime(dict(year=dfgy.year, month=dfgy.month, day=1)).astype(str)
display(dfgy.head())

,latitude,longitude,year,month,cruise,station,cast,day,hour,minute,bottomdepth,maxsampdepth,bottle,pressure,depth,temperature,theta,salinity,salinityf,salinityqc,sigma0,sigma1,sigma2,sigma3,sigma4,gamma,oxygen,oxygenf,oxygenqc,aou,aouf,nitrate,nitratef,nitrateqc,nitrite,nitritef,silicate,silicatef,silicateqc,phosphate,phosphatef,phosphateqc,tco2,tco2f,tco2qc,talk,talkf,talkqc,phts25p0,phts25p0f,phtsinsitutp,phtsinsitutpf,phtsqc,cfc11,pcfc11,cfc11f,cfc11qc,cfc12,pcfc12,cfc12f,cfc12qc,cfc113,pcfc113,cfc113f,cfc113qc,ccl4,pccl4,ccl4f,ccl4qc,sf6,psf6,sf6f,c13,c13f,c14,c14f,c14err,h3,h3f,h3err,he3,he3f,he3err,he,hef,heerr,neon,neonf,neonerr,o18,o18f,toc,tocf,doc,docf,don,donf,tdn,tdnf,chla,chlaf,date
0,-77.3487,-161.842,2011,3,295.0,71.0,1.0,19.0,16.0,10.200000,638.0,636.1,3.0,269.900000,266.600000,-1.766300,-1.772632,34.191480,2.0,1.0,27.522860,32.29270,36.955120,41.51128,45.962540,27.783440,304.880000,2.0,1.0,62.070640,2.0,NaN,9.0,1.0,NaN,9.0,NaN,9.0,1.0,NaN,9.0,1.0,NaN,9.0,1.0,NaN,9.0,1.0,NaN,9.0,NaN,9.0,1.0,NaN,NaN,9.0,1.0,NaN,NaN,9.0,1.0,NaN,NaN,9.0,1.0,NaN,NaN,9.0,1.0,NaN,NaN,9.0,NaN,9.0,NaN,9.0,NaN,9.0,9.0,9.0,NaN,9.0,NaN,NaN,9.0,NaN,NaN,9.0,NaN,NaN,9.0,NaN,9.0,NaN,9.0,NaN,9.0,NaN,9.0,NaN,9.0,2011-03-01
1,-77.3366,-161.086,2011,3,295.0,69.0,1.0,19.0,13.0,7.833333,670.0,668.7,3.5,334.466667,330.666667,-1.781433,-1.789327,34.234283,2.0,1.0,27.558117,32.32810,36.990633,41.54695,45.998333,27.824417,NaN,9.0,1.0,NaN,9.0,NaN,9.0,1.0,NaN,9.0,NaN,9.0,1.0,NaN,9.0,1.0,NaN,9.0,1.0,NaN,9.0,1.0,NaN,9.0,NaN,9.0,1.0,NaN,NaN,9.0,1.0,NaN,NaN,9.0,1.0,NaN,NaN,9.0,1.0,NaN,NaN,9.0,1.0,NaN,NaN,9.0,NaN,9.0,NaN,9.0,NaN,9.0,9.0,9.0,NaN,9.0,NaN,NaN,9.0,NaN,NaN,9.0,NaN,NaN,9.0,NaN,9.0,NaN,9.0,NaN,9.0,NaN,9.0,NaN,9.0,2011-03-01
2,-77.3344,-161.465,2011,3,295.0,70.0,1.0,19.0,14.0,46.200000,674.0,671.3,3.0,264.860000,261.800000,-1.706920,-1.713194,34.192080,2.0,1.0,27.521560,32.28950,36.950100,41.50450,45.954060,27.782080,308.225000,3.4,1.0,57.917525,3.4,NaN,9.0,1.0,NaN,9.0,NaN,9.0,1.0,NaN,9.0,1.0,NaN,9.0,1.0,NaN,9.0,1.0,NaN,9.0,NaN,9.0,1.0,NaN,NaN,9.0,1.0,NaN,NaN,9.0,1.0,NaN,NaN,9.0,1.0,NaN,NaN,9.0,1.0,NaN,NaN,9.0,NaN,9.0,NaN,9.0,NaN,9.0,9.0,9.0,NaN,9.0,NaN,NaN,9.0,NaN,NaN,9.0,NaN,NaN,9.0,NaN,9.0,NaN,9.0,NaN,9.0,NaN,9.0,NaN,9.0,2011-03-01
3,-77.3341,-160.805,2011,3,295.0,68.0,1.0,19.0,11.0,33.400000,541.0,539.7,3.0,273.840000,270.800000,-1.817400,-1.823564,34.165580,2.0,1.0,27.503200,32.27486,36.939060,41.49698,45.949920,27.759660,295.680000,2.0,1.0,71.850100,2.0,NaN,9.0,1.0,NaN,9.0,NaN,9.0,1.0,NaN,9.0,1.0,NaN,9.0,1.0,NaN,9.0,1.0,NaN,9.0,NaN,9.0,1.0,NaN,NaN,9.0,1.0,NaN,NaN,9.0,1.0,NaN,NaN,9.0,1.0,NaN,NaN,9.0,1.0,NaN,NaN,9.0,NaN,9.0,NaN,9.0,NaN,9.0,9.0,9.0,NaN,9.0,NaN,NaN,9.0,NaN,NaN,9.0,NaN,NaN,9.0,NaN,9.0,NaN,9.0,NaN,9.0,NaN,9.0,NaN,9.0,2011-03-01
4,-77.3271,-160.554,2011,3,295.0,67.0,1.0,19.0,10.0,24.666667,440.0,435.4,2.0,229.633333,226.666667,-1.807067,-1.812170,34.081033,2.0,1.0,27.434133,32.20620,36.870800,41.42910,45.882433,27.677333,302.266667,2.0,1.0,65.396767,2.0,NaN,9.0,1.0,NaN,9.0,NaN,9.0,1.0,NaN,9.0,1.0,NaN,9.0,1.0,NaN,9.0,1.0,NaN,9.0,NaN,9.0,1.0,NaN,NaN,9.0,1.0,NaN,NaN,9.0,1.0,NaN,NaN,9.0,1.0,NaN,NaN,9.0,1.0,NaN,NaN,9.0,NaN,9.0,NaN,9.0,NaN,9.0,9.0,9.0,NaN,9.0,NaN,NaN,9.0,NaN,NaN,9.0,NaN,NaN,9.0,NaN,9.0,NaN,9.0,NaN,9.0,NaN,9.0,NaN,9.0,2011-03-01


In [170]:
def get_frames(dfgy, col='date'):
#     dfgg = dfgy.copy().set_index(col)
    
    frames = [{   
        'name':'frame_{}'.format(col_val),
        'data':[{
            'type':'scattermapbox',
            'lat': dfgy.loc[dfgy[col] == col_val, 'latitude'],
            'lon': dfgy.loc[dfgy[col] == col_val, 'longitude'],
            'mode': 'markers',
            'marker': go.scattermapbox.Marker(
                size=log1p(dfgy.loc[dfgy[col] == col_val, 'talk'].replace(nan, 0)),
                opacity=0.5,
                color=dfgy.loc[dfgy[col] == col_val, 'tco2'].replace(-9999, nan),
                showscale=True,
                colorbar={'title':'TCO2', 'titleside':'top', 'thickness':4, 'ticksuffix':' mu mol / kg'},
            ),
            'customdata': np.stack((
               dfgy.loc[dfgy[col] == col_val, 'tco2'],
               dfgy.loc[dfgy[col] == col_val, 'talk'],
               dfgy.loc[dfgy[col] == col_val, 'salinity']
            ), axis=-1),

            'hovertemplate': """
    <extra></extra>
    %{customdata[0]}<br>
    TCO2: %{customdata[0]:.3f} mu mol/kg<br>  
    ALK: %{customdata[1]:.3f}<br>
    SAL: %{customdata[2]:.3f}<br>
    """,
        }]
    } for col_val in dfgy[col].tolist()]
    
    sliders = [{
        'transition':{'duration': 0},
        'x':0.08, 
        'len':0.88,
        'currentvalue':{'font':{'size':15}, 'visible':True, 'xanchor':'center'},  
        'steps':[
            {
                'label':col_val,
                'method':'animate',
                'args':[
                    ['frame_{}'.format(col_val)],
                    {'mode':'immediate', 'frame':{'duration':100, 'redraw': True}, 'transition':{'duration':50}}
                  ],
            } for col_val in dfgy[col].tolist()]
    }]
    
    return frames, sliders

# ALITER
menu_buttons = [{
        "buttons": [
            {
                "args": [None, {"frame": {"duration": 500, "redraw": False},
                                "fromcurrent": True, "transition": {"duration": 300,
                                                                    "easing": "quadratic-in-out"}}],
                "label": "Play",
                "method": "animate"
            },
            {
                "args": [[None], {"frame": {"duration": 0, "redraw": False},
                                  "mode": "immediate",
                                  "transition": {"duration": 0}}],
                "label": "Pause",
                "method": "animate"
            }
        ],
        "direction": "left",
        "pad": {"r": 10, "t": 87},
        "showactive": False,
        "type": "buttons",
        "x": 0.1,
        "xanchor": "right",
        "y": 0,
        "yanchor": "top"
    }
]

In [172]:
frames, sliders = get_frames(dfgy.sort_values(by='date'))

data = frames[0]['data']

# Adding all sliders and play button to the layout
layout = go.Layout(
    sliders=sliders,
    updatemenus=menu_buttons,
    mapbox={
        'accesstoken':mapbox_token,
        'center':{"lat": 37.86, "lon": 2.15},
        'zoom':1.0,
        'style':'light',
    }
)

# Creating the figure
fig = go.Figure(data=data, layout=layout, frames=frames)

# Displaying the figure
fig.show()

# Appendix
Toy examples for scatters and animations

In [102]:
# import plotly.graph_objects as go

# import pandas as pd

# df = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/2011_february_us_airport_traffic.csv')
# df['text'] = df['airport'] + '' + df['city'] + ', ' + df['state'] + '' + 'Arrivals: ' + df['cnt'].astype(str)

# fig = go.Figure(data=go.Scattergeo(
#         lon = df['long'],
#         lat = df['lat'],
#         text = df['text'],
#         mode = 'markers',
#         marker_color = df['cnt'],
#         marker = dict(
#             size = 8,
#             opacity = 0.8,
#             reversescale = True,
#             autocolorscale = False,
#             symbol = 'square',
#             line = dict(
#                 width=1,
#                 color='rgba(102, 102, 102)'
#             ),
#             colorscale = 'Blues',
#             cmin = 0,
#             color = df['cnt'],
#             cmax = df['cnt'].max(),
#             colorbar_title="Major / Minor"        
#         )))

# fig.update_layout(
#         title = 'Most trafficked US airports<br>(Hover for airport names)',
#         geo_scope='usa',
#     )
# fig.show()

In [39]:
# import plotly.express as px
# df = px.data.gapminder().query("year == 2007")
# fig = px.scatter_geo(df, locations="iso_alpha",
#                      color="continent", # which column to use to set the color of markers
#                      hover_name="country", # column added to hover information
#                      size="pop", # size of markers
#                      projection="natural earth")
# fig.show()

In [33]:
# from urllib.request import urlopen
# import json
# with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
#     counties = json.load(response)

# import pandas as pd
# df = pd.read_csv("https://raw.githubusercontent.com/plotly/datasets/master/fips-unemp-16.csv",
#                    dtype={"fips": str})

# import plotly.express as px

# fig = px.choropleth(df, geojson=counties, locations='fips', color='unemp',
#                            color_continuous_scale="Viridis",
#                            range_color=(0, 12),
#                            scope="usa",
#                            labels={'unemp':'unemployment rate'}
#                           )
# fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
# fig.show()

